In [151]:
import os
import polars as pl
pl.Config(tbl_rows=100)
from src.najdi_rok import najdi_rok

In [152]:
# df = pl.read_parquet(os.path.join("data","cnb_ceska_proza.parquet")).rename({'autorstvo_kod':'100_7'})
# df = pl.concat([df, pl.read_parquet(os.path.join("data","cnb_ceska_poezie.parquet"))])
# spisovatelstvo = list(set(df.select(pl.col('100_7')).drop_nulls().to_series().to_list()))
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
# df = df.filter(pl.col("100_7").is_in(spisovatelstvo))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))

In [153]:
df = df.filter(pl.col("rok") >= 1800)

In [154]:
nechcemejetam = [
    "jn19990008769",
    "jx20060515016",
    "jn19981002230",
    "jn19981001737",
    "jn19990210182",
    "jn19990001842",
    "jn19990002786",
    "jn19990004346",
    "jn20020721077",
    "jn19990210513",
    "jn19990005488",
    "jo20000080627",
    "jn19990000171",
    "jn20001005715",
    "jn19981002409",
    "jn20000810141",
    "jn19981002129",
    "jn20001103529",
    "jn20000810032",
    "jn19990001513",
    "jx20040611003",
    "jn19990005499",
    "jn19981002230",
    "jn19990001907",
    "jo2005267810",
    "jo20241218643",
    "jn20000602144",
    "jn19990005454",
    "jn19981228078",
    "xx0010566",
    "jn20010310318",
    "xx0203193"
]

# df = df.filter(~pl.col("100_7").is_in(nechcemejetam))

In [155]:
df.sample(10)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,008,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,655_ind2,655_a,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_e,700_6,700_x,700_f,700_5,700_g,rok
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,str,str,str,list[str],list[str],str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],i64
"""1""","""Zapletal, Ladislav,""","""jk01152111""","[""aut""]","""1924-1999""",null,null,null,null,"""bk196302693""","""971103s1963 xr …","""1""","""0""","""Komenského mapa Moravy /""",null,"""Naps. Ladislav Zapletal ; Příl…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1963
"""1""","""Kipp, Angelika,""","""jn20010309406""","[""aut""]","""1955-""",null,null,null,null,"""cpk20020891065""","""020107s2001 xr a b f 0…","""1""","""0""","""Modrá a biela :""","""obrázky z papiera na okná a na…","""[Angelika Kipp ; z nemeckého o…",null,null,null,null,null,"[""7"", ""7"", … ""9""]","[""příručky"", ""publikace pro děti"", … ""children's literature""]","[""fd133209"", ""fd133156"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2001
"""1""","""Čermáková, Jarmila Hannah,""","""jn20000919545""","[""aut""]","""1945-2024""",null,null,null,null,"""nkc20051417887""","""040930s2005 xr a g 0…","""1""","""0""","""Něžné žně /""",null,"""Jarmila Hannah Čermáková ; [il…",null,null,null,null,null,"[""7"", ""9""]","[""česká poezie"", ""Czech poetry""]","[""fd133958"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2005
"""1""","""Murin, Filip""","""mzk2004252098""","[""aut""]",null,null,null,null,null,"""cpk20041490311""","""041124s2004 xr a e f 0…","""1""","""0""","""Saláty :""","""[pikantní saláty a zálivky pro…","""Filip Murin]""",null,null,null,null,null,"[""7"", ""9""]","[""kuchařské recepty"", ""cookbooks""]","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2004
"""1""","""Macht, Antonín,""","""jk01073059""","[""aut""]","""1903-""",null,null,null,null,"""bk194002173""","""990522s1940 xr e d 0…","""1""","""0""","""Praktický slovník německo-česk…","""Praktisches Wörterbuch Univers…","""Antonín Macht""",null,null,null,null,null,"[""7"", ""7""]","[""frazeologické slovníky"", ""dvojjazyčné překladové slovníky""]","[""fd202923"", ""fd208746""]","[""czenas"", ""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1940
"""1""","""Hellich, Jan,""","""jk01040685""","[""aut""]","""1850-1931""",null,null,null,null,"""bknzdr05808""","""011223s1891 xr c e 0…","""1""","""0""","""P. Matěj Rössler, děkan poděbr…","""nástin životopisný /""","""podává Jan Hellich""",null,null,null,null,null,"[""7""]","[""biografie""]","[""fd131909""]","[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1891
"""1""","""Kratochvíl, Martin,""","""jn99240000525""","[""aut""]","""1972-""",null,null,null,null,"""nkc20001171445""","""010522s2000 xr a g

In [156]:
df = df.drop_nulls(subset=["700_a","700_4","700_7"]).explode(["700_a","700_4","700_7"])

In [157]:
df.group_by([''])

In [158]:
df.select(pl.col(["700_a","700_4","700_7",'245_a'])).sample(10)

700_a,700_4,700_7,245_a
str,str,str,str
"""Kotýk, Jakub,""","""trl""","""xx0207488""","""Zjevení světla :"""
"""Führmann Vízdalová, Ivana,""","""trl""","""jn20000807018""","""Slizké smrti /"""
"""Macků, Zdena""","""ill""",null,"""Víťa Malejev ve škole i doma /"""
"""Stříbrný, Petr""","""trl""","""jn20010310093""","""První mezi rovnými /"""
"""Čapek, Josef,""","""ill""","""jk01021020""","""Štyri rozprávky /"""
"""Walterová, Libuše,""","""aut""","""jo2002106616""","""Pojistná matematika :"""
"""Demant Trobe, Gitte""","""aut""","""mzk2009492047""","""Zranitelnost a intimita :"""
"""Bukovský, Ivan,""","""ill""","""jn19981000360""","""Úsměv dítěte /"""
"""Janotková, Eva""","""aut""","""stk2007405691""","""Termomechanika :"""


In [159]:
from itertools import combinations

In [160]:
def find_collaborations(df):
    # Filter for authors only
    authors = df.filter(pl.col('700_4') == 'aut')
    
    # Group by book title to get authors per book
    books_authors = authors.group_by('245_a').agg(pl.col('700_a').alias('authors'))
    
    # Generate author pairs and count collaborations
    collaborations = []
    for book in books_authors.iter_rows():
        if len(book[1]) > 1:  # Only consider books with multiple authors
            for pair in combinations(sorted(book[1]), 2):
                collaborations.append(pair)
    
    # Convert to dataframe and count frequencies
    collab_df = pl.DataFrame({
        'author1': [p[0] for p in collaborations],
        'author2': [p[1] for p in collaborations]
    })
    
    if len(collab_df) == 0:
        return pl.DataFrame({'author1': [], 'author2': [], 'collaboration_count': []})
    
    return (collab_df.group_by(['author1', 'author2'])
            .count()
            .sort('count', descending=True)
            .rename({'count': 'collaboration_count'}))

In [161]:
find_collaborations(df).filter(pl.col("author1") != pl.col("author2"))

C:\Users\micha\AppData\Local\Temp\ipykernel_12808\3784072569.py:25: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  .count()


author1,author2,collaboration_count
str,str,u32
"""Král, Lukáš""","""Valenta, Tomáš""",1156
"""Malý, Martin""","""Münch, Otto""",897
"""Krupka, Peter,""","""Nechvátalová, Jana,""",857
"""Novotný, Miloš""","""Novák, František""",670
"""Nechvátalová, Jana,""","""Staudková, Hana""",667
"""Krupka, Peter,""","""Staudková, Hana""",667
"""Grünfeldová, Arnoštka,""","""Sarauer, Alois,""",667
"""Sturm, Antonín,""","""Štvánová, Božena""",587
"""Čupera, Jiří,""","""Ždánský, Bronislav""",572


In [162]:
df.filter(pl.col('700_4') == 'aut').group_by(['100_a','100_7','700_a']).agg(pl.col("245_a").n_unique()).sort(by="245_a",descending=True)

100_a,100_7,700_a,245_a
str,str,str,u32
"""David, Petr,""","""xx0006786""","""Soukup, Vladimír,""",165
"""Smíšek, Richard,""","""xx0038627""","""Smíšková, Zuzana""",96
"""Smíšek, Richard,""","""xx0038627""","""Smíšková, Kateřina,""",96
"""Nedbalová, Marie""","""jo2014816080""","""Nedbalová, Josefína""",87
"""Marx, Karl,""","""jn19990005454""","""Engels, Friedrich,""",66
"""Vitochová, Marie,""","""mzk2003202741""","""Kejř, Jindřich,""",58
"""Grimm, Jacob Ludwig Karl,""","""jn19981228078""","""Grimm, Wilhelm Karl,""",54
"""Benešová, Hana,""","""pna2012702164""","""Formáčková, Marie,""",52
"""Benešová, Hana,""","""pna2012702164""","""Fukalová, Lenka""",52


In [163]:
df.filter(pl.col('700_4') == 'ill').group_by(['100_a','100_7','700_a']).agg(pl.col("245_a").n_unique()).sort(by="245_a",descending=True)

100_a,100_7,700_a,245_a
str,str,str,u32
"""Rillich, Ivan,""","""jx20060515016""","""Hofman, Zdeněk,""",122
"""Štíplová, Ljuba,""","""jk01131441""","""Němeček, Jaroslav,""",111
"""Nedbalová, Marie""","""jo2014816080""","""Popprová, Andrea""",87
"""Vojkovský, Rostislav,""","""mzk2003186017""","""Szcurek, Michal""",86
"""Verne, Jules,""","""jn19990008769""","""Benett, Léon,""",73
"""Nedbalová, Marie""","""jo2014816080""","""Poppr, Roman""",68
"""Švandrlík, Miloslav,""","""jk01131832""","""Winter-Neprakta, Jiří,""",68
"""Bass, Eduard,""","""jk01011066""","""Kratochvíl, Zdeněk,""",67
"""Verne, Jules,""","""jn19990008769""","""Roux, George,""",61


In [177]:
df.filter(pl.col('700_4') == 'trl').group_by(['100_a','100_7','700_a']).agg(pl.col("245_a").n_unique()).sort(by="245_a",descending=True)

100_a,100_7,700_a,245_a
str,str,str,u32
"""Unger, Gert F.,""","""jn20001103529""","""Butala, Tomáš,""",311
"""Kirby, John""","""jx20040611003""","""Pavka, Marek,""",182
"""Courths-Mahler, Hedwig,""","""jn19990001513""","""Schönová, Zuzana""",120
"""Scott, William,""","""ola2003188680""","""Butala, Tomáš,""",101
"""Birkner-Mahler, Frieda,""","""jn20000600873""","""Lacinová, Libuše,""",80
"""Unger, Gert F.,""","""jn20001103529""","""Schönová, Zuzana""",80
"""Unger, Gert F.,""","""jn20001103529""","""Sedláčková, Libuše""",77
"""Francis, Dick,""","""jn19990002376""","""Moserová, Jaroslava,""",75
"""Pratchett, Terry,""","""jo20000080627""","""Kantůrek, Jan,""",73


In [185]:
df.group_by("700_4").len().sort(by="len",descending=True)

700_4,len
str,u32
"""aut""",184077
"""trl""",118343
"""ill""",62987
"""edt""",21198
"""aui""",17799
"""pht""",17092
"""art""",14999
null,14040
"""lyr""",13797


In [193]:
df.filter(pl.col('700_4') == 'ill').group_by('700_a').agg(pl.col("245_a").n_unique()).sort(by="245_a",descending=True)

700_a,245_a
str,u32
"""Born, Adolf,""",323
"""Burian, Zdeněk,""",280
"""Zmatlíková, Helena,""",199
"""Němeček, Jaroslav,""",187
"""Bouda, Cyril,""",175
"""Winter-Neprakta, Jiří,""",175
"""Krejčová, Zdeňka,""",172
"""Šplíchal, Antonín,""",166
"""Svolinský, Karel,""",158
